In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


with server

In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=3322)
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Data Migrated to latest version !!!
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
res = client.register(name="a", email="aa@b.org", password="c", password_verify="c")

In [4]:
ds_client = node.login(email="aa@b.org", password="c")

Logged into <test-domain-helm2: High side Domain> as <aa@b.org>


setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

# Setup syft functions

## Dataset

In [5]:
x = ActionObject.from_obj([1, 2])
x_ptr = x.send(ds_client)

## Batch function

In [6]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [7]:
ds_client.code.submit(process_batch)

SyftSuccess: User Code Submitted

## Main function

In [8]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
#     results = [x.wait().get() for x in jobs]
    return 3
#     return sum(results)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [9]:
r = ds_client.code.request_code_execution(process_all)
client.requests[-1].approve(approve_nested=True)

Request approved for domain test-domain-helm2


SyftSuccess: Request f19e660afac5467d8b9a3401978d68da changes applied

In [10]:
job = ds_client.code.process_all(x=x_ptr, blocking=False)
sleep(5)

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft
28/11/23 16:53:22 FUNCTION LOG (e4f4db893879494d9b0b570f728753fe): Launching jobs
28/11/23 16:53:22 FUNCTION LOG (e4f4db893879494d9b0b570f728753fe): starting aggregation
28/11/23 16:53:22 FUNCTION LOG (e4f4db893879494d9b0b570f728753fe): Done


In [11]:
# job.subjobs[0].logs()

In [12]:
# job.subjobs

In [13]:
# client.jobs[0].subjobs[0].logs()

In [14]:
job.wait()

28/11/23 16:53:26 FUNCTION LOG (38062e3886b64aa59f5bcab139ed4544): starting batch 1
28/11/23 16:53:26 FUNCTION LOG (97293057bba2494a83be5cbd1553e8cf): starting batch 2
28/11/23 16:53:27 FUNCTION LOG (38062e3886b64aa59f5bcab139ed4544): done
28/11/23 16:53:27 FUNCTION LOG (97293057bba2494a83be5cbd1553e8cf): done


Pointer:
None

In [15]:
sum([j.wait().get() for j in job.subjobs])

5

In [16]:
node.land()